# TFG FER

Este notebook contiene el código necesario para la creación de un modelo de clasificación de imágenes de rostros humanos en función del estres emocional que presentan. Para ello, se va a utuilzar un dataset de imágenes pero de tipo .npy que contiene las imágenes de los rostros pero con un formato diferente al habitual, ya que las imágenes están en formato de matriz de píxeles. 

Los pasos a seguir para la creación del modelo son los siguientes:

1. Importar las librerías necesarias
2. Cargar el dataset
3. Preprocesamiento de los datos
4. Creación del modelo
5. Entrenamiento del modelo
6. Evaluación del modelo

## 1. Importar las librerías necesarias

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import os
import random

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2024-07-16 18:58:34.147462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 18:58:35.053302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fgalan/miniconda3/lib/:/home/fgalan/miniconda3/lib/:/home/fgalan/miniconda3/envs/myenv/lib/
2024-07-16 18:58:35.053412: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fgalan/miniconda3/lib/:/

## 2. Cargar el dataset

Como se ha mencionado anteriormente, el dataset que se va a utilizar es un archivo .npy que contiene las imágenes de los rostros en formato de matriz de píxeles. Para cargar el dataset, se utiliza la función `np.load()` de la librería NumPy.

Nuestro dataset esta dividido en dos partes, una con las imágenes y otra con las etiquetas. Para la recolección de imágenes se ha utilizado un dataset cuyo escenario experimental consiste en pasos que causan o no causan estrés; lectura de escritura en el idioma nativo, entrevista en el idioma nativo, lectura de escritura en idioma no nativo, entrevista en idioma no nativo.

Por lo tanto las distintas carpetas que forma el dataset son:

- **Native_Language_Script_Reading** (BAJO ESTRES-NEUTRAL)
- **Native_Language_Interview** (BAJO ESTRES)
- **Non_Native_Language_Script_Reading** (ALTO ESTRES-NEUTRAL)
- **Non_Native_Language_Interview** (ALTO ESTRES)

Para procesar archivos .npy de la manera que se ha descrito y asignar las etiquetas correspondientes (como "BAJO ESTRES-NEUTRAL", "BAJO ESTRES", "ALTO ESTRES-NEUTRAL", "ALTO ESTRES") basadas en el nombre de la carpeta en la que se encuentran, se van a seguir los siguientes pasos:

### 2.1 Asignar las etiquetas a las imágenes

Para asignar las etiquetas a las imágenes, se va a utilizar una primero una función que recorra las carpetas del dataset y asigne las etiquetas correspondientes a cada imagen.

In [2]:
def asignar_etiqueta(nombre_carpeta):
    if nombre_carpeta == "Native_Language_Interview":
        return "BAJO ESTRES"
    elif nombre_carpeta == "Native_Language_Script_Reading":
        return "BAJO ESTRES-NEUTRAL"
    elif nombre_carpeta == "Non-native_Language_Interview":
        return "ALTO ESTRES"
    elif nombre_carpeta == "Non-native_Language_Script_Reading":
        return "ALTO ESTRES-NEUTRAL"
    else:
        return None  # Por si el nombre de la carpeta no coincide

### 2.2 Cargar las imágenes y las etiquetas

Para el dataset que se va a utilizar, se van a cargar las imágenes de distintas carpetas, segun la persona que aparece en la imagen, y las etiquetas correspondientes a cada imagen. Dentro de cada carpeta de cada persona, se encuentran las carpetas con las clases de estrés correspondientes, y dentro de cada una de estas carpetas se encuentran las imágenes. Lo que se va a realizar es cargar las imágenes y las etiquetas de cada una de las carpetas de las clases de estrés, y se van a guardar todas las corresponientes a cada persona en un array de numpy. Para asi tener un array de numpy con todas las imágenes y otro con todas las etiquetas.

In [3]:
# Definir la ruta base de tu DATASET
ruta = "DATASET"

# Definir las subcarpetas que corresponden a cada clase
classes = ["Native_Language_Script_Reading", "Native_Language_Interview",
           "Non-native_Language_Script_Reading", "Non-native_Language_Interview"]

# Número deseado de imágenes por clase
num_images_per_class = 800

# Listas para almacenar las imágenes y las etiquetas
imagenes = []
etiquetas = []

# Recorrer cada persona en el dataset
for carpeta_persona in os.listdir(ruta):
    ruta_persona = os.path.join(ruta, carpeta_persona)
    print(ruta_persona)
    if os.path.isdir(ruta_persona):  # Asegurar que es un directorio
        # Recorrer cada clase
        for cls in classes:
            ruta_clase = os.path.join(ruta_persona, cls)
            if os.path.exists(ruta_clase):
                # Lista todas las imágenes (archivos .npy) en la carpeta de la clase
                images = [os.path.join(ruta_clase, img) for img in os.listdir(ruta_clase) if img.endswith('.npy')]            
                # Seleccionar 1000 imágenes aleatorias sin repetición
                if len(images) >= num_images_per_class:
                    selected_images = random.sample(images, num_images_per_class)
                else:
                    selected_images = images  # Si hay menos de 1000, tomar todas
                
                # Añadir las imágenes seleccionadas y sus etiquetas a las listas
                for ruta_imagen in selected_images:
                    imagen = np.load(ruta_imagen, allow_pickle=True)
                    imagenes.append(imagen)
                    etiquetas.append(asignar_etiqueta(cls))
                    
# Contar etiquetas por clase
conteo_etiquetas = {etiqueta: etiquetas.count(etiqueta) for etiqueta in set(etiquetas)}

# Número de etiquetas creadas por clase
for etiqueta, conteo in conteo_etiquetas.items():
    print(f"Etiqueta '{etiqueta}': {conteo} imágenes")



DATASET/05
DATASET/28
DATASET/20
DATASET/29
DATASET/22
DATASET/30
DATASET/10
DATASET/14
DATASET/21
DATASET/02
DATASET/25
DATASET/11
DATASET/15
DATASET/16
DATASET/09
DATASET/33
DATASET/17
DATASET/24
DATASET/32
DATASET/18
DATASET/27
DATASET/03
DATASET/23
DATASET/19
DATASET/13
DATASET/01
DATASET/07
DATASET/08
DATASET/31
DATASET/04
DATASET/12
DATASET/06
Etiqueta 'ALTO ESTRES-NEUTRAL': 25600 imágenes
Etiqueta 'ALTO ESTRES': 25600 imágenes
Etiqueta 'BAJO ESTRES': 25600 imágenes
Etiqueta 'BAJO ESTRES-NEUTRAL': 25600 imágenes


### 2.3 Visualizar las imágenes

Antes de continuar con el preprocesamiento de los datos, se van a visualizar algunas de las imágenes del dataset para comprobar que se han cargado correctamente.

In [4]:
# Comprobar que se han cargado las imágenes
print("Número de imágenes cargadas:", len(imagenes))
print("Número de etiquetas cargadas:", len(etiquetas))


Número de imágenes cargadas: 102400
Número de etiquetas cargadas: 102400


Vamos a visualizar el número de imágenes que hay en cada categoría

In [5]:
# Contar cuántas imágenes hay de cada etiqueta
from collections import Counter

conteo = Counter(etiquetas)
print("Número de imágenes por etiqueta:", conteo)  
    

Número de imágenes por etiqueta: Counter({'BAJO ESTRES-NEUTRAL': 25600, 'BAJO ESTRES': 25600, 'ALTO ESTRES-NEUTRAL': 25600, 'ALTO ESTRES': 25600})


### 2.4 Preprocesamiento de los datos

Una vez que se han cargado las imágenes y las etiquetas, se van a preprocesar los datos para poder utilizarlos en el modelo de clasificación. Para ello, se van a seguir los siguientes pasos:

- Normalizar las imágenes
- Dividir los datos en conjuntos de entrenamiento y prueba
- Codificar las etiquetas

#### 2.4.1 Normalizar las imágenes

In [6]:
# Convertir la lista de imágenes a un array de NumPy para facilitar el procesamiento
imagenes = np.array(imagenes, dtype="float32")
print(imagenes.shape)

# Normalizar las imágenes dividiendo los valores de los píxeles por 255
imagenes = imagenes / 255
print(imagenes.shape)

(102400, 256, 7, 7)
(102400, 256, 7, 7)


#### 2.4.2 Dividir los datos en conjuntos de entrenamiento y prueba

Dividimos el conjunto de entrenamiento y el conjunto de pruebas para evaluar el rendimiento

In [7]:
from sklearn.model_selection import train_test_split

# Convertir las etiquetas a un array de NumPy
etiquetas = np.array(etiquetas)
# print(etiquetas)

# Dividir los datos en datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42, stratify=etiquetas)


#### 2.4.3 Codificar las etiquetas

In [8]:
# Para modelos de clasificación, especialmente en Keras/TensorFlow, las etiquetas deben estar codificadas de manera que cada etiqueta sea un vector binario. Esto se conoce como codificación one-hot
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

# Codificar las etiquetas
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Apliar la codificación one-hot
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Comprobar las etiquetas codificadas
print(y_train_categorical)
print(y_test_categorical)

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


## 3. Creación del modelo

Una vez que se han preprocesado los datos, se va a crear el modelo de clasificación de imágenes de rostros humanos en función del estrés emocional que presentan. Para ello, se va a utilizar una red neuronal densa capas totalmente conectadas.

### 3.1 Definición de los modelos

En este apartado se van a explicar los modelos que se van a utilizar para la clasificación de las imágenes. Todos los modelos que se muestran en este apartado son redes neuronales densas.
Todos estos modelos se han elegido debido a que a lo largo del tiempo se han utilizado para diferentes objetivos de clasificación y han obtenido buenos resultados.

#### 3.1.1 Modelo Prueba

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

model = Sequential([
    Flatten(input_shape=(256, 7, 7)),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # Asume 4 clases como en tu ejemplo
])

2024-07-16 18:59:33.996811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-16 18:59:33.997102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-16 18:59:34.015167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-16 18:59:34.015418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-16 18:59:34.015641: I tensorflow/compiler/xla/stream_executo

#### Compilar y entrenar el modelo

In [10]:
# Antes de entrenar el modelo, debes compilarlo, especificando la función de pérdida y el optimizador que utilizarás.

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train_categorical, batch_size=32, epochs=10, validation_split=0.1)

2024-07-16 18:59:35.371737: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 3699376128 exceeds 10% of free system memory.
2024-07-16 18:59:37.508070: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 3699376128 exceeds 10% of free system memory.


Epoch 1/10


2024-07-16 18:59:40.222222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-07-16 18:59:40.232978: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0692d5d250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-16 18:59:40.232995: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-07-16 18:59:40.233000: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-07-16 18:59:40.245253: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-16 18:59:40.413802: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is 

2304/2304 [==============================] - 7s 2ms/step - loss: 0.1899 - accuracy: 0.9331 - val_loss: 0.0650 - val_accuracy: 0.9790
Epoch 2/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0611 - accuracy: 0.9802 - val_loss: 0.0374 - val_accuracy: 0.9873
Epoch 3/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0435 - accuracy: 0.9854 - val_loss: 0.0273 - val_accuracy: 0.9913
Epoch 4/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0338 - accuracy: 0.9886 - val_loss: 0.0250 - val_accuracy: 0.9910
Epoch 5/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0268 - accuracy: 0.9913 - val_loss: 0.0254 - val_accuracy: 0.9911
Epoch 6/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0247 - accuracy: 0.9916 - val_loss: 0.0196 - val_accuracy: 0.9930
Epoch 7/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0211 - accuracy: 0.9930 - val_loss: 0.0213 - val_accuracy: 0.9924
Epo

#### Validación del modelo


In [11]:
# Validación del modelo
test_loss, test_acc = model.evaluate(X_test, y_test_categorical)
print('Precisión en el conjunto de prueba:', test_acc)
print('Pérdida en el conjunto de prueba:', test_loss)

2024-07-16 19:00:32.751584: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1027604480 exceeds 10% of free system memory.
2024-07-16 19:00:33.415522: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1027604480 exceeds 10% of free system memory.


640/640 [==============================] - 1s 2ms/step - loss: 0.0238 - accuracy: 0.9935
Precisión en el conjunto de prueba: 0.993457019329071
Pérdida en el conjunto de prueba: 0.023823585361242294


#### 3.1.2 Modelo LeNet

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Asumiendo que tus datos son características extraídas (256 características por cada una de las 7x7 "imágenes")
model_LeNet = Sequential([
    Flatten(input_shape=(256, 7, 7)), # Aplanamos las características para hacerlas compatibles con capas densas
    Dense(512, activation='relu'), # Aumentamos la dimensión. Ajusta según necesidad.
    Dropout(0.5), # Ayuda a prevenir el sobreajuste
    Dense(120, activation='relu'), # Capa densa con 120 nodos como en LeNet
    Dense(84, activation='relu'), # Capa densa con 84 nodos como en LeNet
    Dense(4, activation='softmax') # Capa de salida para 4 clases
])


#### Compilar y entrenar el modelo

In [13]:
# Compilar el modelo
model_LeNet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model_LeNet.fit(X_train, y_train_categorical, batch_size=32, epochs=10, validation_split=0.1)

2024-07-16 19:00:35.028859: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 3699376128 exceeds 10% of free system memory.


Epoch 1/10
2304/2304 [==============================] - 8s 3ms/step - loss: 0.1808 - accuracy: 0.9277 - val_loss: 0.0513 - val_accuracy: 0.9818
Epoch 2/10
2304/2304 [==============================] - 6s 3ms/step - loss: 0.0887 - accuracy: 0.9682 - val_loss: 0.0377 - val_accuracy: 0.9871
Epoch 3/10
2304/2304 [==============================] - 6s 3ms/step - loss: 0.0750 - accuracy: 0.9731 - val_loss: 0.0396 - val_accuracy: 0.9851
Epoch 4/10
2304/2304 [==============================] - 6s 2ms/step - loss: 0.0743 - accuracy: 0.9723 - val_loss: 0.0319 - val_accuracy: 0.9884
Epoch 5/10
2304/2304 [==============================] - 6s 3ms/step - loss: 0.0614 - accuracy: 0.9780 - val_loss: 0.0274 - val_accuracy: 0.9905
Epoch 6/10
2304/2304 [==============================] - 6s 3ms/step - loss: 0.0675 - accuracy: 0.9747 - val_loss: 0.0283 - val_accuracy: 0.9905
Epoch 7/10
2304/2304 [==============================] - 5s 2ms/step - loss: 0.0732 - accuracy: 0.9713 - val_loss: 0.0452 - val_accuracy:

#### Validación del modelo

In [14]:
# Validación del modelo
test_loss, test_acc = model_LeNet.evaluate(X_test, y_test_categorical)
print('Precisión en el conjunto de prueba:', test_acc)
print('Pérdida en el conjunto de prueba:', test_loss)

640/640 [==============================] - 1s 2ms/step - loss: 0.0304 - accuracy: 0.9917
Precisión en el conjunto de prueba: 0.991650402545929
Pérdida en el conjunto de prueba: 0.03039696253836155
